In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from keras.utils.np_utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
import time

In [ ]:
# ~ Data preprocessing section ~
# Save the training data from fashion-mnist_test.csv to the "train" DataFrame
train = pd.read_csv("drive/MyDrive/Colab Notebooks/fashion-mnist_train.csv")
# Save the testing data from fashion-mnist_test.csv to the "test" DataFrame
test = pd.read_csv("drive/MyDrive/Colab Notebooks/fashion-mnist_test.csv")
# epx = Diffent number of epochs
epx = [10,25,50,75,100]

In [ ]:
# In this section, we chose to train the best models with diffrent number of epochs
# We chose to keep 1 model per CNN type, 1 for a 1-Layer CNN, 1 for a 2-Layer CNN, 1 for a 3-Layer CNN, 1 for a 4-Layer CNN

# Define the 1st Convolutional Neural Network, the best of our 1-Layer CNN tested so far.
name1 = '1_NN_256'
cnn_model_1 = Sequential([], name = name1)
# 1st Convolutional Layer
cnn_model_1.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_1.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model_1.add(BatchNormalization())
cnn_model_1.add(Flatten())
cnn_model_1.add(Dense(128, activation='relu'))
cnn_model_1.add(Dense(10, activation='softmax'))

# Define the 2nd Convolutional Neural Network, the best of our 2-Layer CNN tested so far.
name2 = '2_NN_128_128'
cnn_model_2 = Sequential([], name = name2)
# 1st Convolutional Layer
cnn_model_2.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_2.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model_2.add(BatchNormalization())
# 2nd Convolutional Layer
cnn_model_2.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_2.add(BatchNormalization())
cnn_model_2.add(Flatten())
cnn_model_2.add(Dense(128, activation='relu'))
cnn_model_2.add(Dense(10, activation='softmax'))

# Define the 3rd Convolutional Neural Network, the best of our 3-Layer CNN tested so far.
name3 = '3_NN_128_256_256'
cnn_model_3 = Sequential([], name = name3)
# 1st Convolutional Layer
cnn_model_3.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_3.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model_3.add(BatchNormalization())
# 2nd Convolutional Layer
cnn_model_3.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_3.add(BatchNormalization())
# 3rd Convolutional Layer
cnn_model_3.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_3.add(Dense(256, activation='relu'))
cnn_model_3.add(Flatten())
cnn_model_3.add(Dense(128, activation='relu'))
cnn_model_3.add(Dense(10, activation='softmax'))

# Define the 4th Convolutional Neural Network, the best of our 4-Layer CNN tested so far.
name4 = '4_NN_128_128_128_128'
cnn_model_4 = Sequential([],name = name4)
# 1st Convolutional Layer
cnn_model_4.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_4.add(BatchNormalization())
# 2nd Convolutional Layer
cnn_model_4.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_4.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model_4.add(BatchNormalization())
# 3rd Convolutional Layer
cnn_model_4.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_4.add(Dense(128, activation='relu'))
# 4th Convolutional Layer
cnn_model_4.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn_model_4.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model_4.add(BatchNormalization())
cnn_model_4.add(Flatten())
cnn_model_4.add(Dense(128, activation='relu'))
cnn_model_4.add(Dense(10, activation='softmax'))


models = [cnn_model_1,cnn_model_2,cnn_model_3,cnn_model_4]
names = [name1, name2, name3, name4]
# Στα dictionaries models_accur, models_loss αποθηκέυουμε για κάθε μοντέλο 
#τα αντιστοιχα test accur test loss για καθε αριθμό sample , 
#δηλαδη models_accur = {1ΝΝ_256:[accur για 1000δειγματα,accur για 5000 δειγματα, κοκ ],2ΝΝ_64_128[...],...}
models_accur = {name1:[],name2:[],name3:[],name4:[]}
models_loss = {name1:[],name2:[],name3:[],name4:[]}

In [ ]:
# Define a dictionary
history_dict = {}
# With the following function, we train each model for a diffrent number of epochs 
def bullet_4(model,name):
    for i in epx:
        # Data preprocessing ~ same as before
        Y_train = train['label'].values
        X_train = train.drop(labels = ['label'], axis = 1)
        Y_test = test['label'].values
        X_test = test.drop(labels = ['label'], axis = 1)
        X_train = X_train / 255.0
        X_test = X_test / 255.0
        X_train = X_train.values.reshape(-1,28,28,1)
        X_test = X_test.values.reshape(-1,28,28,1)
        # convert to one-hot-encoding
        Y_train = to_categorical(Y_train, num_classes = 10)
        # convert to one-hot-encoding
        Y_test = to_categorical(Y_test, num_classes = 10)
        # Define our optimizer
        optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        # Compile the selected model
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        # Define our datagenerator object
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # dimesion reduction
            rotation_range=0.1,  # randomly rotate images in the range
            zoom_range = 0.1, # Randomly zoom image
            width_shift_range=0.1,  # randomly shift images horizontally
            height_shift_range=0.1,  # randomly shift images vertically
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # Generate data with the parameters given above, of our training DataFrame
        datagen.fit(X_train)
        # Initialize split method
        X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 2)

        
        start=time.time()
        history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                                  shuffle=True, epochs=i, 
                                  validation_data = (X_val, Y_val),verbose = 0, 
                                  steps_per_epoch=X_train.shape[0] // 128)
        end=time.time()
        history_dict[model.name] = history
        # Evaluate the selected model
        score_model = model.evaluate(X_test, Y_test)
        print(i)
        # Print the evaluation results
        print('Test loss:', score_model[0])
        print('Test accuracy:', score_model[1])
        # Print the time that every model did for training
        print('Execution Time:',end-start)
        # Save the evaluation of each model
        models_accur[name].append(score_model[0])
        models_loss[name].append(score_model[1])
        

In [ ]:
# By calling the function, the training begins for every model and a diffrent number of epochs each time
for i in range(0,4):
    bullet_4(models[i],names[i])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


313/313 [==============================] - 1s 3ms/step - loss: 0.3067 - accuracy: 0.8897
10
Test loss: 0.30669087171554565
Test accuracy: 0.8896999955177307
Execution Time: 125.7446620464325
313/313 [==============================] - 1s 3ms/step - loss: 0.2843 - accuracy: 0.9063
25
Test loss: 0.2843060791492462
Test accuracy: 0.9063000082969666
Execution Time: 312.77123832702637
313/313 [==============================] - 1s 3ms/step - loss: 0.3146 - accuracy: 0.9137
50
Test loss: 0.31458187103271484
Test accuracy: 0.9136999845504761
Execution Time: 617.2789211273193
313/313 [==============================] - 1s 3ms/step - loss: 0.3394 - accuracy: 0.9151
75
Test loss: 0.3394123911857605
Test accuracy: 0.9150999784469604
Execution Time: 910.8562443256378
313/313 [==============================] - 1s 3ms/step - loss: 0.3633 - accuracy: 0.9194
100
Test loss: 0.3633403480052948
Test accuracy: 0.9193999767303467
Execution Time: 1189.2050967216492
313/313 [==============================] - 1s

In [ ]:
# ~ Plotting Section ~
# Plotting techniques same as before
plt.figure(1,figsize=(25,15))
ticks = [10,15,25,50]
for history in history_dict:
    train_acc = history_dict[history].history['accuracy']
    val_acc = history_dict[history].history['val_accuracy']
    loss = history_dict[history].history['loss']
    val_loss = history_dict[history].history['val_loss']

    plt.subplot(2,1,1)
    plt.xticks(ticks=ticks)
    plt.title("Validation Loss results")
    plt.plot(val_loss, label=""+history)
    plt.xlabel("Epochs")
    #plt.ylim(0,1)
    plt.xlim(0,5)
    plt.ylabel("Validation Loss")
    plt.legend()

    plt.subplot(2,1,2)
    plt.xticks(ticks=ticks)
    plt.title("Validation Loss results")
    plt.plot(loss, label=""+history)
    plt.xlabel("Epochs")
    plt.xlim(0,5)
    plt.ylabel("Training Loss ")
    plt.legend()
plt.show()